# 소개
슈팅스타! 팀 프로젝트를 colab환경에서 간단히 테스트 해볼 수 있는 ipynb 파일 입니다.
___

## 사전 준비
- data 폴더엔 질문-문서.jsonl 파일과, 학습된 모델.pth 파일을 넣어 주시면 됩니다.
- vectordb 폴더는 자동생성되고, 만약 있는걸 사용하신다면 하단 main함수에 알맞은 경로를 넣어서 불러와서 사용하면 됩니다.
- ipynb파일 경로를 %cd를 통해 지정해주고 (예시) `%cd '/content/drive/MyDrive/Colab Notebooks/yeardream/shootingstar_test'`
    - 해당 폴더에 .env를 만들고 안에 'OPENAI_API_KEY'='sk-...'
- 최하단에 main.py에 들어가는 인자를 알맞게 수정
- 특히 is_first가 True일시 문서 임베딩을 처음부터 다시 하는데다가, 덮어쓰는게 아니라 추가로 넣어버리기 때문에 해당모델을 하는게 아니라면 True로 설정하시면 안됩니다.
    - 이 값을 True로 하는 경우 = 모델을 바꾸거나, workspace를 변경했을때 True
___


## main.py 부분 사용법
- 문서데이터를 처음부터 임베딩 하는게 아니라면 굳이 gpu런타임을 할 필요 없습니다.
- 모두 실행후 최하단에 input 입력창이 생기면 질문을 넣고 답변을 받을 수 있습니다.
___
## eval.py 부분 사용법
- main.py 이전셀을 실행후 실행하면 됩니다
- main.py를 돌리고 있는 중이라면 'exit'를 입력해 빠져나오거나, 직접 중지해주고 실행해주세요
- model_paths = ["data/museum_5epochs.pth", ""]
    - 평가할 모델의 경로를 순서대로 적어줍니다 (workspace와 순서가 같아야함)
    - ""은 klue/bert-base 모델을 적용합니다 (dpr X)
- workspaces = ["vectordb/museum_5epochs", "vectordb/bert-base"]
    - 해당 모델로 임베딩한 index.bin이 있는 경로를 지정해줍니다.
    - 여기서 '/' 다음 부분이 csv파일에 모델이름으로 기록되게 됩니다.




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 자신의 폴더 위치에 맞게 아래 경로 변경해야 함
%cd '/content/drive/MyDrive/Colab Notebooks/슈팅스타/shootingstar_test'

/content/drive/MyDrive/Colab Notebooks/슈팅스타/shootingstar_test


In [3]:
!pip install python-dotenv docarray vectordb langchain openai
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━

In [4]:
import os
import json
from dotenv import load_dotenv
from typing import Optional, Literal
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.pyplot as plt

from docarray import BaseDoc, DocList
from docarray.typing import NdArray
from vectordb import InMemoryExactNNVectorDB

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, BertModel
from kobert_tokenizer import KoBERTTokenizer


from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage, ChatMessage
from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks import get_openai_callback

import textwrap


load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
# llmvdb/doc.py
class ToyDoc(BaseDoc):
    text: str = ""
    context_embedding: Optional[NdArray[768]]
    question: str = ""
    question_embedding: Optional[NdArray[768]]
    tit_id: str = ""
    ctx_id: str = ""


In [6]:
# llmvdb/customdataset.py

class CustomDataset(Dataset):
    def __init__(self, file_path, max_lines=None):
        self.documents_data = []
        seen_ctx_ids = set()
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                data = json.loads(line)
                ctx_id = data.get("ctx_id")

                if ctx_id not in seen_ctx_ids:
                    self.documents_data.append(data)
                    seen_ctx_ids.add(ctx_id)

                if max_lines and len(self.documents_data) >= max_lines:
                    break

    def __len__(self):
        return len(self.documents_data)

    def __getitem__(self, idx):
        data = self.documents_data[idx]
        text = f'{data.get("title", "")}\n{data.get("context", "")}{data.get("description","")}'
        return {
            "text": text,
            "question": data.get("question", ""),
            "ctx_id": data.get("ctx_id"),
            "tit_id": data.get("tit_id"),
        }


class EvalCustomDataset(CustomDataset):
    def __init__(self, file_path):
        self.documents_data = []
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                data = json.loads(line)
                self.documents_data.append(data)

In [7]:
# llmvdb/embedding.py

class HuggingFaceEmbedding:
    def __init__(self, model_name: str = "klue/bert-base", use_gpu: bool = True):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() and use_gpu else "cpu"
        )
        if model_name == "skt/kobert-base-v1":
            self.tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")
            self.model = BertModel.from_pretrained("skt/kobert-base-v1").to(self.device)
            pass
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModel.from_pretrained(model_name).to(self.device)
        print(f"====={self.device}를 사용해서 임베딩합니다=====")

    def get_embedding(self, prompt):
        if isinstance(prompt, str):
            prompt = [prompt]

        inputs = self.tokenizer(
            prompt, padding=True, truncation=True, return_tensors="pt", max_length=512
        )
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            model_output = self.model(**inputs)
        sentence_embeddings = self.mean_pooling(model_output, inputs["attention_mask"])
        return sentence_embeddings.cpu()

    @staticmethod
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        )
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
            input_mask_expanded.sum(1), min=1e-9
        )


class DPRTextEmbedding(HuggingFaceEmbedding):
    def __init__(
        self,
        mode: Literal["passage", "question"],
        model_path: str = "./data/kmrc_mrc.pth",
        model_name: str = "klue/bert-base",
    ):
        if mode not in ["passage", "question"]:
            raise ValueError("Mode must be 'passage' or 'question'")
        super().__init__(model_name)
        self.mode = mode
        self.model_dict = {}
        self.load_model = torch.load(model_path, map_location=torch.device("cpu"))
        for key in self.load_model.keys():
            if key.startswith(f"module.{self.mode}_encoder."):
                self.model_dict[
                    key.replace(f"module.{self.mode}_encoder.", "")
                ] = self.load_model[key]
        self.model.load_state_dict(self.model_dict, strict=True)


In [8]:
# llmvdb/langchain.py

class LangChain:
    def __init__(self, api_token=None, instruction=None, callbacks=None, verbose=False):
        self.api_token = api_token or os.getenv("OPENAI_API_KEY") or None
        if self.api_token is None:
            raise ValueError("Open API 키가 필요합니다")
        self.instruction = instruction
        self.callbacks = callbacks
        self.verbose = verbose
        self.streaming = False if self.callbacks is None else True
        self.model = "gpt-3.5-turbo-1106"
        self.llm = ChatOpenAI(
            model=self.model,
            temperature=0.3,
            streaming=self.streaming,
            callbacks=self.callbacks,
        )
        self.system_message = f"{self.instruction}\n 주어지는 참고용 문서를 바탕으로 사용자의 질문에 답변해줘. 사용자의 질문을 자세히 분석해서 문서를 참고해 질문의 답변을 해줘\n 만약 문서에서 질문에 대한 답변을 찾을 수 없으면 반드시 '참고할 수 있는 문서가 없습니다.' 라고 말하고, 이후에 너가 아는 정보를 말해줘"
        self.history_memory = []
        self.initial_history_memory = [SystemMessage(content=self.system_message)]

    def append_limit_length(self, human_message, ai_message):
        new_pair_len = len(human_message.content) + len(ai_message.content)

        while (
            sum(len(msg.content) for msg in self.history_memory) + new_pair_len > 3000
        ):
            self.history_memory.pop(0)
            self.history_memory.pop(0)
        self.history_memory.append(human_message)
        self.history_memory.append(ai_message)

    def call(self, prompt: str, document: str) -> str:
        with get_openai_callback() as cb:
            response = self.llm(
                self.initial_history_memory
                + self.history_memory
                + [
                    HumanMessage(
                        content=f"질문:\n{prompt}\n\n ### 참고용 문서(질문과 관련 없으면 절대 참고하지 않기):\n{document}\n"
                    )
                ]
            )

        if self.verbose:
            print(cb)

        response = response.content
        print(response)

        self.append_limit_length(
            HumanMessage(content=prompt), AIMessage(content=response)
        )
        return response

    def set_callbacks(self, callbacks):
        self.llm = ChatOpenAI(
            model=self.model,
            temperature=0.3,
            streaming=self.streaming,
            callbacks=callbacks,
        )


In [9]:
# llmvdb/__init__.py

class Llmvdb:
    def __init__(
        self,
        embedding=None,
        llm=None,
        verbose: bool = False,
        file_path=None,
        workspace: Optional[str] = None,
        threshold: float = 0.7,
        top_k: int = 3,
    ):
        self.embedding = embedding
        self.llm = llm
        self.verbose = verbose
        self.workspace = workspace
        self.file_path = file_path
        self.threshold = threshold
        self.top_k = top_k

        self.db = InMemoryExactNNVectorDB[ToyDoc](workspace=self.workspace)

    def custom_collate_fn(self, batch):
        texts = [item["text"] for item in batch]
        questions = [item["question"] for item in batch]
        ctx_ids = [item["ctx_id"] for item in batch]
        tit_ids = [item["tit_id"] for item in batch]
        batched_data = []
        for i in range(len(batch)):
            batched_data.append(
                {
                    "text": texts[i],
                    "question": questions[i],
                    "ctx_id": ctx_ids[i],
                    "tit_id": tit_ids[i],
                }
            )
        return batched_data

    def initialize_db(self):
        dataset = CustomDataset(self.file_path)
        dataloader = DataLoader(
            dataset, batch_size=64, shuffle=False, collate_fn=self.custom_collate_fn
        )
        doc_list = []

        for batch in tqdm(dataloader, desc="Processing dataset embedding"):
            texts = [data["text"] for data in batch if data["text"].strip() != ""]
            context_embeddings = self.embedding.get_embedding(texts)

            for j, data in enumerate(batch):
                if data["text"].strip() != "":
                    doc_list.append(
                        ToyDoc(
                            text=data["text"],
                            context_embedding=context_embeddings[j],
                            question=data["question"],
                            tit_id=data["tit_id"],
                            ctx_id=data["ctx_id"],
                        )
                    )

        self.db.index(inputs=DocList[ToyDoc](doc_list))
        self.db.persist()

    def retrieve_document(self, prompt):
        query = ToyDoc(
            text=prompt, context_embedding=self.embedding.get_embedding(prompt)
        )
        search_parameters = {"search_field": "context_embedding"}
        results = self.db.search(
            inputs=DocList[ToyDoc]([query]),
            parameters=search_parameters,
            limit=self.top_k,
        )

        input_document = ""
        over_threshold_indices = [
            idx for idx, value in enumerate(results[0].scores) if value > self.threshold
        ]

        if self.verbose:
            # print(results[0].matches[0])
            # print(results[0].matches.ctx_id)
            print(results[0].text, results[0].scores)
            print(f"threshold를 넘는 index : {over_threshold_indices}")

            # 만약 threshold 0.8을 넘는게 있고 그 개수가 k개보다 적다면 전부 retrieve
        if 1 <= len(over_threshold_indices) < self.top_k:
            for index in over_threshold_indices:  # top-k (k=3)
                input_document += (
                    "#문서" + str(index) + "\n" + results[0].matches[index].text + "\n"
                )

        # 만약 threshold 0.8을 넘는게 있고 그 개수가 k개보다 많다면 top-k만 retrieve
        elif len(over_threshold_indices) >= self.top_k:
            for index in range(self.top_k):  # top-k (k=3)
                input_document += (
                    "#문서" + str(index) + "\n" + results[0].matches[index].text + "\n"
                )

        # 만약 threshold 0.8을 넘는게 없다면 top-1만
        elif len(over_threshold_indices) == 0:
            input_document += "#문서\n" + results[0].matches[0].text + "\n"

        if self.verbose:
            print("================아래 문서를 참고합니다================")
            print(input_document)
            print("======================================================")

        return input_document

    def generate_response(self, prompt):
        input_document = self.retrieve_document(prompt)
        completion = self.llm.call(prompt, input_document)
        return completion

    def change_embedding(self, new_embedding):
        self.embedding = new_embedding

    def evaluate_model(self, target_model):
        dataset = EvalCustomDataset("data/test.jsonl")
        dataloader = DataLoader(
            dataset, batch_size=32, shuffle=False, collate_fn=self.custom_collate_fn
        )
        question_list = []
        for batch in tqdm(dataloader, desc=f"embedding..{target_model}"):
            question = [data["question"] for data in batch]
            question_embedding = self.embedding.get_embedding(question)
            for idx, data in enumerate(batch):
                question_list.append(
                    ToyDoc(
                        question=data["question"],
                        question_embedding=question_embedding[idx],
                        ctx_id=data["ctx_id"],
                        tit_id=data["tit_id"],
                    )
                )
        question_len = len(question_list)
        correct_counts = {
            1: 0,
            2: 0,
            3: 0,
            5: 0,
            7: 0,
            10: 0,
            20: 0,
            50: 0,
            "model": target_model,
            "criteria": "ctx_id",
        }
        correct_counts_tit_id = correct_counts.copy()
        correct_counts_tit_id["criteria"] = "tit_id"

        top_k_keys = sorted([k for k in correct_counts.keys() if isinstance(k, int)])

        for q in tqdm(question_list, desc=f"search..{target_model}"):
            search_query = ToyDoc(
                question=q.question,
                context_embedding=q.question_embedding,
                ctx_id=q.ctx_id,
                tit_id=q.tit_id,
            )
            for top_k in top_k_keys:
                search_parameters = {"search_field": "context_embedding"}
                search_results = self.db.search(
                    inputs=DocList[ToyDoc]([search_query]),
                    parameters=search_parameters,
                    limit=top_k,
                )

                ctx_id_matched = False
                for match in search_results[0].matches:
                    if match.ctx_id == search_query.ctx_id:
                        correct_counts[top_k] += 1
                        correct_counts_tit_id[top_k] += 1
                        ctx_id_matched = True
                        break

                if not ctx_id_matched:
                    for match in search_results[0].matches:
                        if match.tit_id == search_query.tit_id:
                            correct_counts_tit_id[top_k] += 1
                            break

        for key in top_k_keys:
            correct_counts[key] = correct_counts[key] / question_len
            correct_counts_tit_id[key] = correct_counts_tit_id[key] / question_len

        return correct_counts, correct_counts_tit_id


In [11]:
# main.py
class CLIHandler(BaseCallbackHandler):
    def __init__(self):
        self.text=''

    def on_llm_new_token(self, token:str, **kwargs) -> None:
        self.text += token

def main(
    data_file_path: str,
    workspace: str,
    model_path: str,
    model_name: str='klue/bert-base',
    is_dpr: bool = False,
    is_first: bool = False,
):
    cli_handler = CLIHandler()

    if is_dpr:
        embedding = DPRTextEmbedding("passage", model_path, model_name)
        question_embedding = DPRTextEmbedding("question", model_path, model_name)
    else:
        embedding = HuggingFaceEmbedding(model_name)

    llm = LangChain(callbacks=[cli_handler])

    llmvdb = Llmvdb(
        embedding,
        llm,
        file_path=data_file_path,
        workspace=workspace,
        verbose=True,  # False로 설정시 터미널에 정보 출력 안됨
        threshold=0.1,
        top_k=5,
        )
    if is_first:
        llmvdb.initialize_db()  # vectordb저장, 처음에 한번만 실행

        # is_dpr = True 이면 question embedding으로 변경
    if is_dpr:
        llmvdb.change_embedding(question_embedding)

    while True:
        user_input = input('user: ')
        if user_input.lower() == 'exit':
            break
        try:
            response = llmvdb.generate_response(user_input)
            wrapped_response = textwrap.fill(response, width=55)
            print('AI: ', wrapped_response)
        except Exception as e:
            print(f'응답 생성중 오류 발생: {e}')

if __name__ == "__main__":
    main(
        # 질문-문서 데이터셋 경로
        data_file_path='data/train.jsonl',

        # 임베딩된 데이터가 저장되는(되어있는) 경로
        workspace='vectordb/museum_skt_kobert',

        #학습된 dpr모델(.pth파일)의 경로
        model_path='data/museum_skt_kobert.pth',

        # 기반이 되는 모델
        model_name='skt/kobert-base-v1',

        # DPR 모델 사용 여부
        is_dpr=True,

        # 처음 실행 여부
        is_first=False,
        # 주의 : 이 값을 True로 하는 경우 = 모델을 바꾸거나, workspace를 변경했을때 True
        # 처음 폴더를 받은 상태에서 돌려보기만 할땐 False로 둬도 됨!
    )



tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

=====cuda를 사용해서 임베딩합니다=====
=====cuda를 사용해서 임베딩합니다=====


WARNING - docarray - Index file does not exist: /content/drive/MyDrive/Colab Notebooks/슈팅스타/shootingstar_test/vectordb/museum_monologg_kobert_5ep/InMemoryExactNNIndexer[ToyDoc][ToyDocWithMatchesAndScores]/index.bin. Initializing empty InMemoryExactNNIndex.


Processing dataset embedding:   0%|          | 0/48 [00:00<?, ?it/s]

user: 마패에 대해서 알려줘


마패에 대해서 알려줘 [0.04271072521805763, 0.04242512956261635, 0.04232857748866081, 0.04229998588562012, 0.04228636622428894]
threshold를 넘는 index : []
================아래 문서를 참고합니다================
#문서
자치통감사정전훈의
“훈의만큼 상세하고 정밀한 책은 세상에 없을 것이다. 나는 우리나라의 훈의가 가장 우수하다고 생각한다.”

참고할 수 있는 문서가 없습니다. 혹시 마패가 어떤 것인지에 대해 더 자세한 정보를 알려주실 수 있나요?
AI:  참고할 수 있는 문서가 없습니다. 혹시 마패가 어떤 것인지에 대해 더 자세한 정보를 알려주실 수
있나요?
user: exit


In [ ]:
# eval.py
!mkdir -p eval



def eval_model(
    model_paths: list,
    workspaces: list,
    model_names: list,
    is_first: bool = False,
):
    if not is_first:
        if os.path.exists("eval/model_performance.csv"):
            df = pd.read_csv("eval/model_performance.csv")
            existed_models = set("vectordb/" + df["model"])

    model_performance = []
    for model_path, workspace, model_name in zip(model_paths, workspaces, model_names):
        target_model = workspace.split("/")[-1]
        if not is_first and workspace in existed_models:
            print(f"{target_model}의 평가 결과는 이미 저장되어 있으므로 넘어갑니다.")
            continue
        print(model_path, workspace)
        if model_path:
            embedding = DPRTextEmbedding("question", model_path, model_name)
        else:
            print("====== klue/bert-base =====")
            embedding = HuggingFaceEmbedding(model_name)
        vdb = Llmvdb(
            embedding,
            workspace=workspace,
            verbose=True,
        )

        accuracy, accuracy_tit = vdb.evaluate_model(target_model)
        for accuracy_dict in [accuracy, accuracy_tit]:
            new_accuracy_dict = {
                f"Top_{k}": v for k, v in accuracy_dict.items() if isinstance(k, int)
            }
            for k in ["model", "question_len", "criteria"]:
                if k in accuracy_dict:
                    new_accuracy_dict[k] = accuracy_dict[k]
            model_performance.append(new_accuracy_dict)

    df = pd.DataFrame(model_performance)
    if is_first:
        df.to_csv(f"eval/model_performance.csv", index=False, encoding="utf-8")
    elif not df.empty:
        df.to_csv(
            f"eval/model_performance.csv",
            mode="a",
            index=False,
            header=False,
            encoding="utf-8",
        )
    return df


class GraphMaker:
    def __init__(self, df: pd.DataFrame):
        # df = df.drop(columns="Top_50")
        self.df = df
        self.top_k_labels = [label for label in df.columns if label.startswith("Top_")]
        self.x_values = [int(label.split("_")[1]) for label in self.top_k_labels]
        self.model_colors = {
            model: color
            for model, color in zip(
                df["model"].unique(), plt.rcParams["axes.prop_cycle"].by_key()["color"]
            )
        }

    def make_graph(self, criteria=None):
        plt.figure(figsize=(12, 8))

        if criteria:
            self.plot_criteria(criteria, linestyle="solid")
        else:
            self.plot_criteria("ctx_id", linestyle="solid")
            self.plot_criteria("tit_id", linestyle="dotted")
        plt.title(
            f"Model-wise Top_k Accuracy ({'ctx_id & tit_id ' if not criteria else criteria})"
        )
        self.finalize_plot()

    def plot_criteria(self, criteria, linestyle):
        for model in self.df["model"].unique():
            subset = self.df[
                (self.df["model"] == model) & (self.df["criteria"] == criteria)
            ]
            y_values = subset[self.top_k_labels].iloc[0]
            plt.plot(
                self.x_values,
                y_values,
                label=f"{model} ({criteria})",
                linestyle=linestyle,
                color=self.model_colors[model],
            )

    def finalize_plot(self):
        plt.xlabel("Top_k")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.xticks(self.x_values, self.top_k_labels, rotation=45)
        plt.grid(True)
        plt.show()


if __name__ == "__main__":
    # 평가할 모델의 경로를 순서대로 적어줍니다 (workspace와 순서가 같아야함)
    # ""은 klue/bert-base 모델을 적용합니다 (dpr X)
    model_paths = [
        "",
        "data/museum_5epochs.pth",
        "data/merged_pn_5ep.pth",
        "data/museum_kdpr.pth",  # aihub_5epochs
        "data/museum_monologg_kobert.pth",  # 5ep
    ]

    # 해당 모델로 임베딩한 index.bin이 있는 경로를 지정해줍니다.
    # 여기서 '/' 다음 부분이 csv파일에 모델이름으로 기록되게 됩니다.
    workspaces = [
        "vectordb/bert-base",
        "vectordb/museum_5epochs",
        "vectordb/merged_pn_5ep",
        "vectordb/aihub_5epochs",
        "vectordb/museum_monologg_kobert_5ep",
    ]

    # 기반 모델 이름(기본: klue/bert-base)
    model_names = [
        "klue/bert-base",
        "klue/bert-base",
        "klue/bert-base",
        "klue/bert-base",
        "monologg/kobert",
    ]
    # eval/model_performance.csv 파일이 있다면 False로, False이면 이전에 했던 평가는 저장된값을 사용함
    is_first = False

    df = eval_model(model_paths, workspaces, model_names, is_first)

    df = pd.read_csv(f"eval/model_performance.csv")
    print(df[df["criteria"] == "ctx_id"])
    print(df[df["criteria"] == "tit_id"])
    graph_maker = GraphMaker(df)
    graph_maker.make_graph("ctx_id")
    graph_maker.make_graph("tit_id")
    graph_maker.make_graph()
